In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
MODEL = "llama3"

In [2]:
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

In [6]:
print(model.invoke("Tell me a joke"))

Why couldn't the bicycle stand up by itself?

Because it was two-tired!

(Sorry, I know it's a bit of a "dad" joke, but I hope it brought a smile to your face!)


In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# pipe the model output as an input to the parser using chain
chain = model | parser  # define the chain
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?"

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("fog_computing_paper.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='2DP-FHS: 2D Pareto Optimi zed Fog Head Selection for  \nMultiple  EEG H ealthcare  Data Analysis and \nComputations  \nSri Harsha Kurra1, Rama Krushna Rath1, Sreeja S. R.1 \n \n1 Indian Institute of Information Technology Sri City,  \nChittoor, Andhra Pradesh, India  \n \nAbstra ct. In recent years, the increase in heal thcare data  demands the  adoption  \nof fog computing for  fast processing and analysis of large data volumes. Fog \ncomputing enables  real-time data computations that serves closely to healthcare \ndevices. This study propo ses the analysis and computations of EEG healthcare \ndata using Pareto optimization -based fog computing. The proposed 2DP -FHS \ntechnique selects a fog head among heterogeneous fog devices to manage  the \nEEG data within the fog layer. Additionally, an alternat ive fog head is \ndesignated to ensure continuous operation within the fog layer . The study  \nconsider s various  scenarios and unbiased fog devices in the  si

In [9]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply with "I don't know".
Context: {context}
Question: {question}
"""

# building up the prompt using the template
prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't answer the question, reply with "I don't know".
Context: Here is some context
Question: Here is a question



In [10]:
chain.input_schema.schema()

{'title': 'OllamaInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'tit

In [11]:
chain = prompt | model | parser  
chain.invoke({
    "context": "The name I was given was Harsha",
    "question": "What is my name?",
})

'Your name is Harsha.'

In [12]:
# using a vector database to convert all the pages of the pdf to embeddings and then use the embeddings to find the most similar text
# we can later compare those embeddings with the embeddings of the prompt and the question to find the most similar text
from langchain_community.vectorstores import DocArrayInMemorySearch
# creating vector store from the pages in memory
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

/Users/sriharsha/Desktop/Clg/RAG/local-model/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [13]:
# retriever is a component of langchain that is used to retrieve the most similar text / context based on the input
retriever = vectorstore.as_retriever()
retriever.invoke("Fog Computing")
#? retriever returns the 4 top documents that are most relevant to the retrieved term
#? sorted in the order of importance
#? this retriever is basically used to provide context to the model

[Document(page_content='2.1   Architectural View   \nThe proposed architecture consists of two layers: a device layer and a fog layer; as \nshown in Fig.  1. The devi ce layer consists of multiple EEG devices which are \nindependent of each other. In the fog layer, heterogeneous fog devices are connected \nto the FH.   \nThe functions  of the device layer are  to: \n\uf0b7 Sense  brain signals  of patients using multiple EEG sensors  \n\uf0b7 Divide  the continuous raw EEG signal into chunks  \n\uf0b7 Transmit  the chunks as output files to the fog layer  \nThe functions  of the fog layer are  to: \n\uf0b7 Receive the EEG data files from the device layer  \n\uf0b7 Select  one of the fog devices as FH  \n\uf0b7 Collaborative and distributed proces sing of the tasks among fog devices  \n\uf0b7 Acknowledge  the resp onses  \n \n                   \n  \nFig. 1. Proposed  2DP-FHS architecture .', metadata={'source': 'fog_computing_paper.pdf', 'page': 2}),
 Document(page_content='Case  II: I

In [16]:
# response from Llama2
from operator import itemgetter
chain = (
  {
    "context": itemgetter("question") | retriever, #? the context is going to come from the retriever
    "question": itemgetter("question"),
  } | 
  prompt | model | parser
)
#? the retriever requires the question that is invoking the chain to provide context to the model
chain.invoke({ "question": "what is fog computing" }) 

'Fog computing is a distributed computing paradigm that involves processing data closer to the source of the data, rather than sending it to a centralized cloud or data center for processing. In contrast to traditional cloud computing, where data is sent to a remote server for processing, fog computing brings computing resources closer to the edge of the network, typically within a few kilometers of the data source. This can help reduce latency and improve real-time processing capabilities, making it particularly useful for applications that require fast processing times, such as IoT devices, autonomous vehicles, and smart cities.\n\nFog computing typically involves a distributed architecture with multiple nodes located at different points in the network, each node performing specific tasks and communicating with other nodes to achieve a common goal. This allows for more efficient use of resources and can help ensure that data is processed closer to where it is generated, reducing late

In [14]:
# response from Llama3
from operator import itemgetter
chain = (
  {
    "context": itemgetter("question") | retriever, #? the context is going to come from the retriever
    "question": itemgetter("question"),
  } | 
  prompt | model | parser
)
#? the retriever requires the question that is invoking the chain to provide context to the model
chain.invoke({ "question": "what is fog computing" }) 

'Based on the context provided, Fog Computing can be defined as a decentralized architecture that provides data processing and analytics closer to where it is generated, reducing latency and improving real-time decision-making. It involves multiple devices (M) such as EEG devices and N number of fog devices that process and analyze data in real-time, enabling faster and more efficient processing of data.'

In [17]:
# response from Llama2
questions = [
  "What is the main objective of this work?",
  "What are the possible challenges in fog computing?",
  "What are the possible solutions to the challenges in fog computing?",
  "What is the future scope of fog computing?",
  "How is fog computing better than cloud computing in solving their problem?",
  "What problem are the authors trying to solve?",
  "What is the main contribution of this work?",
  "What are the limitations of this work?",
  "What are the possible future directions of this work?",
  "What is the significance of this work?",
]

for question in questions: 
  print("Question: ", question)
  print(f"Answer: ", chain.invoke({ 'question': question }))
  print()

Question:  What is the main objective of this work?
Answer:  The main objective of this work is to propose a novel fog computing-based approach for EEG healthcare systems, which can efficiently process and analyze large volumes of EEG data in real-time. The authors aim to address the challenges of high latency and bandwidth consumption in traditional cloud-based EEG systems by leveraging the computing resources of multiple heterogeneous fog devices. Additionally, the authors aim to optimize the selection of fog heads and fog resources for optimal performance in EEG healthcare systems using Pareto optimization.

Question:  What are the possible challenges in fog computing?
Answer:  Based on the provided documents, some of the possible challenges in fog computing include:

1. Data Management Problems: The vast amount of data generated by IoT devices and sensors can cause challenges in managing and processing the data in real-time. (Document 3, Page 5)
2. Processor and Memory Limitations:

In [15]:
# response from Llama3
questions = [
  "What is the main objective of this work?",
  "What are the possible challenges in fog computing?",
  "What are the possible solutions to the challenges in fog computing?",
  "What is the future scope of fog computing?",
  "How is fog computing better than cloud computing in solving their problem?",
  "What problem are the authors trying to solve?",
  "What is the main contribution of this work?",
  "What are the limitations of this work?",
  "What are the possible future directions of this work?",
  "What is the significance of this work?",
]

for question in questions: 
  print("Question: ", question)
  print(f"Answer: ", chain.invoke({ 'question': question }))
  print()

Question:  What is the main objective of this work?
Answer:  The main objective of this work is to propose a methodology for optimal fog head selection in an EEG healthcare system using Pareto optimization, with the goal of minimizing delay and bandwidth while ensuring uninterrupted operation.

Question:  What are the possible challenges in fog computing?
Answer:  I don't know. The provided context does not mention specific challenges in fog computing. It appears to be a paper about fog computing, discussing topics such as synthetic fog dataset, selection of fog head and alternate fog head, and average delay analysis with increase in number of EEG devices and fog devices. To answer the question, I would need more information or a different context.

Question:  What are the possible solutions to the challenges in fog computing?
Answer:  I don't know. The provided context appears to be a research paper on fog computing, discussing various aspects such as edge computing, IoT systems, clus

In [18]:
# response from Llama2
for s in chain.stream({"question": "How is fog computing better than cloud computing in solving their problem?"}):
  print(s, end="", flush=True)

Fog computing is better than cloud computing in solving the problem of EEG healthcare systems for several reasons:

1. Latency: Fog computing reduces latency by processing data closer to the source, which is critical in real-time healthcare analysis. Cloud computing can take longer to process data due to its distance from the source and higher network latency.
2. Scalability: Fog computing supports scalability better than cloud computing as it can handle a large volume of data from multiple EEG devices. In contrast, cloud computing can become overwhelmed with too much data, leading to slower processing times.
3. Heterogeneity: Fog computing supports heterogeneity by allowing for multiple fog devices with different configurations to process data in parallel and distributed decision-making processes. Cloud computing often lacks this level of flexibility and customization.
4. Cost-effectiveness: Fog computing can be more cost-effective than cloud computing as it leverages existing infrast

In [16]:
# response from Llama3
for s in chain.stream({"question": "How is fog computing better than cloud computing in solving their problem?"}):
  print(s, end="", flush=True)

Based on the context, fog computing is better than cloud computing in solving the problem of real-time healthcare analysis with minimum latency and faster response time. This is because fog computing provides:

* Real-time data analysis with minimum latency
* Faster response time
* Scalability and heterogeneity, which helps in processing huge volumes of EEG data in a distributed manner

These advantages are not explicitly mentioned as being better than cloud computing, but they imply that fog computing can provide a more suitable solution for real-time healthcare analysis compared to cloud computing.